# Exercise: Build a simple logistic regression model

In this exercise, we will fit a simple logistic regression model that will try to predict the chance of an avalanche. 

Recall that logistic regression fits an s-shaped curve to data, rather than a straight line, and we can use this to calculate a probability of a binary outcome.

## Data visualization

Let's start this exercise by loading in and having a look at our data:

In [1]:
import pandas

#Import the data from the .csv file
dataset = pandas.read_csv('Data/avalanche.csv', delimiter="\t")

#Let's have a look at the data
dataset

,Unnamed: 0,avalanche,no_visitors,surface_hoar,fresh_thickness,wind,weak_layers,tracked_out
0,0,1,4,3.900508,4.194255,6,9,0
1,1,0,9,1.477586,2.280187,30,0,0
2,2,1,3,3.236594,1.111226,8,8,1
3,3,1,0,3.244283,4.827641,12,10,0
4,4,1,2,5.196741,-0.738915,4,9,1
...,...,...,...,...,...,...,...,...
1090,1090,1,1,2.182905,1.587954,3,9,0
1091,1091,1,2,3.718231,5.904870,28,0,0
1092,1092,1,8,7.037647,5.219775,34,3,0
1093,1093,1,2,2.449889,2.816349,37,10,1


## Data Exploration

The `avalanche` field is our target. A value of `1` means that an avalanche did occur at the conditions described by the features, whereas a value of `0` means no avalanche hapenned. Since our targets can only be `0` or `1` we call this a *binary classification* model.

Now let's plot the relationships between each feature and the target values. That helps us understand which features are more likely to influence the results:

In [2]:
import graphing # custom graphing code. See our GitHub repo for details

graphing.box_and_whisker(dataset, label_x="avalanche", label_y="surface_hoar", show=True)
graphing.box_and_whisker(dataset, label_x="avalanche", label_y="fresh_thickness", show=True)
graphing.box_and_whisker(dataset, label_x="avalanche", label_y="weak_layers", show=True)
graphing.box_and_whisker(dataset, label_x="avalanche", label_y="no_visitors")

We can notice that:

- For `fresh_thickness` the outcomes are very similar. This means that variations in their values are not strongly correlated with the results.

- Variations in values for `weak_layers` and `no_visitors`, seem to correlate with a larger number of `avalanche` results, and thus we should assign more importance to these features.

The differences between avalanche and non-avalanche days are small and there is not one clear driver of issues. Weak layers looks like a good starting point as it is related to the widest variation in results.

## Building a simple logistic regression model

We will now built and train a model to predict the chance of an avalanche happening based __solely__ on the number of weak layers of snow:

In [3]:
# Here we import a function that splits datasets according to a given ratio
from sklearn.model_selection import train_test_split

# Split the dataset in an 70/30 train/test ratio. 
train, test = train_test_split(dataset, test_size=0.3, random_state=2)
print(train.shape)
print(test.shape)

(766, 8)
(329, 8)


OK, lets train our model, using the `train` dataset we've just created (notice that `weak_layers` will be the only feature used to determine the outcome):

In [4]:
import statsmodels.formula.api as smf
import graphing # custom graphing code. See our GitHub repo for details

# Perform logistic regression.
model = smf.logit("avalanche ~ weak_layers", train).fit()

print("Model trained")

Optimization terminated successfully.
         Current function value: 0.450020
         Iterations 6
Model trained
C:\Users\leere\Anaconda3\envs\ml_crash_course\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



After training, we can print a model summary with very detailed information:

In [5]:
print(model.summary())

                           Logit Regression Results                           
Dep. Variable:              avalanche   No. Observations:                  766
Model:                          Logit   Df Residuals:                      764
Method:                           MLE   Df Model:                            1
Date:                Thu, 17 Jun 2021   Pseudo R-squ.:                 0.09675
Time:                        20:43:34   Log-Likelihood:                -344.72
converged:                       True   LL-Null:                       -381.64
Covariance Type:            nonrobust   LLR p-value:                 8.439e-18
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.2366      0.155      1.525      0.127      -0.067       0.541
weak_layers     0.2694      0.034      7.955      0.000       0.203       0.336


Notice that the positive coefficient for `weak_layers` means that a higher value means a higher likelyhood for an avalanche.


## Using our model

We can now use our trained model to make predictions and estimate probabilities.

Let's pick the first three occurrences in our `test` set and print the probability of an avalanche for each one of them

In [6]:
# predict to get a probability

# get first 3 samples from dataset
samples = test["weak_layers"][:3]

# use the model to get predictions as possibilities
estimated_probabilities = model.predict(samples)

# Print results for each sample
for sample, pred in list(zip(samples,estimated_probabilities)):
    print(f"A weak_layer with value {sample} yields a {pred * 100:.2f}% chance of an avalanche.")


A weak_layer with value 5 yields a 82.97% chance of an avalanche.
A weak_layer with value 4 yields a 78.82% chance of an avalanche.
A weak_layer with value 7 yields a 89.31% chance of an avalanche.


Let's plot out model to understand this:

In [7]:
# plot the model
# Show a graph of the result
fig = graphing.scatter_2D(  train,  
                            label_x="weak_layers", 
                            label_y="avalanche",
                            size_multiplier=2.5,
                            trendline=lambda x: model.predict(pandas.DataFrame({"weak_layers": x}))
                            )
fig.show()


This plot can look a bit confusing!

If the occurrence of an avalanche is determined by `weak_layers`, why do we have `0`s and `1`s for the same value of `x`?

Remember that in our dataset we have different days - and outcomes -  with the same value for `weak_layers`. This feature is a strong predictor of an avalanche happening, but it's not the only one affecting the outcome.

The red trendline plots the function of the __probability__ of an avalanche over the number of weak layers; Notice that the more weak layers, the more likely an avalanche will happen.

## Classification or decision thresholds
To return a binary category (`True` = "avalanche", `False` = "no avalanche") we need to define a *Classification Threshold* value. Any probability above that threshold is returned as the positive category, whereas values below it will be returned as the negative category.

Let's see what happens if set our threshold to `0.5` (meaning that our model will return `True` whenever it calculates a chance above 50% of an avalanche happening):

In [8]:
# threshold to get an absolute value
threshold = 0.5

# Add classification to the samples we used before
for sample, pred in list(zip(samples,estimated_probabilities)):
    print(f"A weak_layer with value {sample} yields a chance of {pred * 100:.2f}% of an avalanche. Classification = {pred > threshold}")



A weak_layer with value 5 yields a chance of 82.97% of an avalanche. Classification = True
A weak_layer with value 4 yields a chance of 78.82% of an avalanche. Classification = True
A weak_layer with value 7 yields a chance of 89.31% of an avalanche. Classification = True


Note that a `0.5` threshold is just a starting point that needs to be tuned depending on the data we are trying to classify.

## Performance on test set
Now let's use our `test` dataset to perform a quick evaluation on how the model did. For now, we'll just look at how often we correctly predicted if there would be an avalanche or not

In [12]:
import numpy as np

# Classify the mdel predictions using the threshold
predictions = model.predict(test) > threshold

# Compare the predictions to the actual outcomes in the dataset
accuracy = np.average(predictions == test.avalanche)

# Print the evaluation
print(f"The model correctly predicted outcomes {accuracy * 100:.2f}% of time.")

The model correctly predicted outcomes 78.72% of time.


Avalanches are hard to pick but we're doing well. It's hard to tell exactly what kind of mistakes our model is making, though. We'll focus on this in the next exercise.

## Summary

In this unit we covered the following topics:
 - Using Data Exploration to understand which features have a stronger correlation to the outcomes
 - Building a simple Logistic Regression model
 - Using the trained model to predict _probabilities_
 - Using *thresholds* to map probabilities to binary classes
 - How to use a test set to measure the model's performance
